<a href="https://colab.research.google.com/github/akram01Br/VQA-project/blob/main/VQA_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!pip install transformers → تثبت مكتبة النصوص باش تفهم السؤال

!pip install torchvision → تثبت مكتبة الصور باش تحلل الصورة



In [ ]:
!pip install transformers
!pip install torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

2. نستورد المكتبات الأساسية:
python
Copy
Edit


PIL هي مكتبة باش تقرأ وتتعامل مع الصور في بايثون. تستعمل Image باش تفتح الصورة وتقراها.



from transformers import BertTokenizer, BertModel
تستورد من مكتبة transformers:

BertTokenizer: باش تقسم السؤال النصي لكلمات صغيرة (tokens) وتنظمو باش يفهمهم نموذج BERT.

BertModel: هو نموذج BERT نفسه إلي يعطيك تمثيل رقمي (embedding) للنص.



In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
from transformers import BertTokenizer, BertModel

# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)


Using device: cuda


4. نجهز موديل ResNet pretrained لمعالجة الصورة:
python
Copy
Edit


In [ ]:
# تحميل موديل ResNet50 pretrained
resnet = models.resnet50(pretrained=True)
resnet = resnet.to(device)
resnet.eval()

# نحذف آخر طبقة تصنيف باش نأخذ الميزات فقط
modules = list(resnet.children())[:-1]  # حذف الطبقة الأخيرة
resnet_feat = torch.nn.Sequential(*modules)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 180MB/s]


5. نحضر preprocess للصور:

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
# 6. تحميل موديل BERT لمعالجة النص
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)
bert_model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
# 7. دالة لاستخراج ميزات الصورة
def extract_image_features(image_path):
    image = Image.open(image_path).convert('RGB')
    img_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        features = resnet_feat(img_tensor)
    features = features.view(features.size(0), -1)  # flatten
    return features
    # 8. دالة لاستخراج ميزات النص
def extract_text_features(question):
    inputs = tokenizer(question, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token
    return cls_embedding

In [ ]:
img_features = extract_image_features('ronaldo.jpg')
text_features = extract_text_features("best football player all the time")

print("Image features shape:", img_features.shape)
print("Text features shape:", text_features.shape)


Image features shape: torch.Size([1, 2048])
Text features shape: torch.Size([1, 768])


In [ ]:
import urllib.request
img_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/d7/Cristiano_Ronaldo_playing_for_Al_Nassr_FC_against_Persepolis%2C_September_2023_%28cropped%29.jpg/800px-Cristiano_Ronaldo_playing_for_Al_Nassr_FC_against_Persepolis%2C_September_2023_%28cropped%29.jpg'
urllib.request.urlretrieve(img_url, 'ronaldo.jpg')

('ronaldo.jpg', <http.client.HTTPMessage at 0x7df4d2021b50>)

In [ ]:
import torch.nn as nn

class VQAModel(nn.Module):
    def __init__(self):
        super(VQAModel, self).__init__()
        # دمج ميزات ResNet (2048) و BERT (768)
        self.fc1 = nn.Linear(2048 + 768, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 3)  # 3 أنواع إجابات

    def forward(self, img_feat, text_feat):
        combined = torch.cat((img_feat, text_feat), dim=1)  # دمج الميزات
        x = self.fc1(combined)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# إنشاء النموذج ونقلو للجهاز (CPU ولا GPU)
model = VQAModel().to(device)
model.eval()

# استعمل النموذج للتوقع (inference)
with torch.no_grad():
    output = model(img_features, text_features)
    probs = torch.softmax(output, dim=1)
    print("Output probabilities:", probs)

# باش تطبع الاحتمالات لكل تصنيف
_, predicted = torch.max(probs, 1)
labels = ["Yes", "No", "Don't know"]
print("Predicted answer:", labels[predicted.item()])


Output probabilities: tensor([[0.3550, 0.3537, 0.2913]], device='cuda:0')
Predicted answer: Yes
